In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [ ]:
spark = SparkSession.builder.appName("readfromjson").master("local[2]").getOrCreate()

In [ ]:
static = spark.read.json("activity-data/")
dataSchema = static.schema

In [ ]:
static.printSchema()

In [ ]:
# COMMAND ----------

streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json("activity-data")

In [ ]:
# COMMAND ----------

activityCounts = streaming.groupBy("gt").count()

In [ ]:
# COMMAND ----------

activityQuery = activityCounts.writeStream.queryName("activity_counts")\
  .format("memory").outputMode("complete")\
  .start()

In [ ]:
from time import sleep
for x in range(20):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

In [ ]:
activityQuery.stop()